In [121]:
import pandas as pd
import statistics as s

users = pd.read_csv('users.csv', sep='\t', encoding="koi8-r")
users.columns = ['user_id', 'email', 'geo']

log = pd.read_csv("log.csv", header=None)
log.columns = ['user_id', 'time', 'bet', 'win']

log[['bet', 'win']] = log[['bet', 'win']].fillna(0)
log['net'] = log['win']-log['bet']

log.time = log.time.str.strip('[')
log['time'] = pd.to_datetime(log['time'], format='%Y-%m-%d %H:%M:%S')

# Приведем признак user_id к одному формату в обоих датасетах  
users.user_id = users.user_id.apply(lambda x: x.lower())

# Избавимся от ошибок в user_id  
log = log[log.user_id != '#error']  
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])

merged_df = pd.merge(users, log, on='user_id')

40.10.1

In [122]:
merged_df.groupby('user_id')['net'].sum().median()

1986.0

40.10.2

In [123]:
users_with_bets = merged_df[merged_df['bet']>0]['user_id'].unique()

merged_df[(merged_df['bet'] == 0) & (merged_df['user_id'].isin(users_with_bets))].groupby('user_id')['bet'].count().mean()

5.05

40.10.3

In [124]:
first_visit_without_bet = merged_df[merged_df['bet'] == 0].groupby('user_id')['time'].min()
first_bet = merged_df[merged_df['bet'] > 0].groupby('user_id')['time'].min()

user_idle_time = (first_bet - first_visit_without_bet)

In [125]:
user_idle_time[user_idle_time>pd.Timedelta('-1us')].mean().days

49

40.10.4

In [126]:
merged_df.groupby('geo')['win'].sum().sort_values(ascending=False)

geo
Москва             11959741.0
Воронеж              184338.0
Санкт-Петербург      151007.0
Казань                97806.0
Ярославль             97441.0
Ижевск                84895.0
Красноярск            84767.0
Арзангелтск           74375.0
Пермь                 67734.0
Хабаровск             65459.0
Краснодар             62718.0
Ставрополь            46003.0
Екатеринбург          36682.0
Тюмень                 4701.0
Name: win, dtype: float64

40.10.5

In [127]:
merged_df[merged_df['bet'] > 0].groupby('geo')['bet'].mean().max()/merged_df[merged_df['bet'] > 0].groupby('geo')['bet'].mean().min()

127.81602335164835

40.10.6

In [128]:
merged_df.groupby('geo')['user_id'].count()

geo
Арзангелтск         96
Воронеж             88
Екатеринбург        49
Ижевск              61
Казань              66
Краснодар           86
Красноярск          56
Москва              61
Пермь               55
Санкт-Петербург    115
Ставрополь          36
Тюмень              32
Хабаровск           60
Ярославль           89
Name: user_id, dtype: int64